Capston Project - The Battle of Neighborhoods
Best Coffe Shop Location in San Francisco

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes

import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [12]:
response = requests.get("http://www.healthysf.org/bdi/outcomes/zipmap.htm")
soup = BeautifulSoup(response.text, "lxml")
table = soup.find_all("table")
df = pd.read_html(str(table))
df = pd.DataFrame(df[4])
df.columns = df.iloc[0]
# Sort neighborhoods by population
df.sort_values(by=['Population (Census 2000)'], inplace=True)
# We see that Chinatown is the smallest neighborhood and Inner Mission/Bernal Heights  is the largest neighborhood
df = df.iloc[1:-1, :-1]
df_sf = df
df_sf.head()

,Zip Code,Neighborhood
3,94107,Potrero Hill
17,94127,St. Francis Wood/Miraloma/West Portal
15,94123,Marina
2,94103,South of Market
19,94132,Lake Merced


In [13]:
!pip install uszipcode
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in df_sf.iterrows():
    zipcode = search.by_zipcode(row["Zip Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

df_sf["Latitude"] = latitude
df_sf["Longitude"] = longitude

df_sf.dropna(inplace=True)
df_sf.head()

     |████████████████████████████████| 378kB 23.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 30.4MB/s eta 0:00:01
     |████████████████████████████████| 122kB 25.6MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/2b/dc/d5/e4bc37df332b7a88022bc79d6cdccb79068fdc1620d86afc3a
Successfully built autopep8
Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


,Zip Code,Neighborhood,Latitude,Longitude
3,94107,Potrero Hill,37.77,-122.39
17,94127,St. Francis Wood/Miraloma/West Portal,37.73,-122.46
15,94123,Marina,37.80,-122.44
2,94103,South of Market,37.78,-122.41
19,94132,Lake Merced,37.72,-122.48


In [14]:
df.shape

(20, 4)

In [15]:
df.head(10)

,Zip Code,Neighborhood,Latitude,Longitude
3,94107,Potrero Hill,37.77,-122.39
17,94127,St. Francis Wood/Miraloma/West Portal,37.73,-122.46
15,94123,Marina,37.80,-122.44
2,94103,South of Market,37.78,-122.41
19,94132,Lake Merced,37.72,-122.48
20,94133,North Beach/Chinatown,37.80,-122.44
18,94131,Twin Peaks-Glen Park,37.75,-122.44
1,94102,Hayes Valley/Tenderloin/North of Market,37.78,-122.42
8,94114,Castro/Noe Valley,37.76,-122.44
9,94115,Western Addition/Japantown,37.79,-122.44


Map of SF

In [17]:
latitude = 37.7792808
longitude = -122.4192363
sf_map = folium.Map(location = [latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_sf['Latitude'], df_sf['Longitude'], df_sf['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(sf_map)  
    
sf_map

In [19]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Localidad', 
                  'Localidad Latitude', 
                  'Localidad Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

Find Nearby Coffee Shops

In [20]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZKLPKG2Y5UVSDQ4JVVZTSUUW1XHSGOZ341KBJ0KOTV5UYTLE'
CLIENT_SECRET = 'S2JFSK4R1OSSIMPGFPYSUMZHLL0VLONDUBLDYCULGKQGMTH2'
VERSION = '20181020'

In [23]:
sf_venues_coffee = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1e0931735')
sf_venues_coffee.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Potrero Hill,37.77,-122.39,Starbucks,37.771196,-122.389364,Coffee Shop
1,Potrero Hill,37.77,-122.39,Spro Coffeelab,37.770835,-122.391400,Coffee Shop
2,Potrero Hill,37.77,-122.39,Starbucks,37.777057,-122.393659,Coffee Shop
3,Potrero Hill,37.77,-122.39,Chat’s Roasting Company,37.762384,-122.398287,Coffee Shop
4,Potrero Hill,37.77,-122.39,Réveille Coffee Co.,37.773309,-122.391508,Coffee Shop


In [25]:
sf_venues_coffee.shape

(483, 7)

Find Nearby High Schools

In [26]:
sf_venues_highschools = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13d941735')
sf_venues_highschools.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Potrero Hill,37.77,-122.39,International Studies Academy,37.761814,-122.401238,High School
1,Potrero Hill,37.77,-122.39,ISAH,37.761955,-122.401107,High School
2,St. Francis Wood/Miraloma/West Portal,37.73,-122.46,Archbishop Riordan High School,37.727942,-122.453227,High School
3,St. Francis Wood/Miraloma/West Portal,37.73,-122.46,Stratford School,37.723121,-122.461527,High School
4,St. Francis Wood/Miraloma/West Portal,37.73,-122.46,ARHS RSP Coordinator Office,37.725999,-122.449298,High School


In [27]:
sf_venues_highschools.shape

(94, 7)

Find Nearby Universities

In [30]:
sf_venues_uni = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ae941735')
sf_venues_uni.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Potrero Hill,37.77,-122.39,UCSF Mission Bay,37.769028,-122.395933,Medical School
1,Potrero Hill,37.77,-122.39,UCSF Mission Bay - Sandler Neurosciences Bldg,37.768982,-122.391918,University
2,Potrero Hill,37.77,-122.39,Mission Hall Hub,37.767685,-122.390748,College Library
3,Potrero Hill,37.77,-122.39,Acadamy Of Art - Sculpture Building,37.774545,-122.399022,College Arts Building
4,Potrero Hill,37.77,-122.39,UCSF Mission Bay OR,37.766147,-122.390055,University


In [31]:
sf_venues_uni.shape

(67, 7)

Find Nearby Offices

In [32]:
sf_venues_office = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
sf_venues_office.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Potrero Hill,37.77,-122.39,Thrive City,37.767912,-122.388709,Event Space
1,Potrero Hill,37.77,-122.39,Cisco Meraki,37.770252,-122.387388,Tech Startup
2,Potrero Hill,37.77,-122.39,UCSF Mission Bay,37.769028,-122.395933,Medical School
3,Potrero Hill,37.77,-122.39,UCSF Benioff Children's Hospital San Francisco,37.764813,-122.390091,Hospital
4,Potrero Hill,37.77,-122.39,Oracle Park Parking Lot A,37.775371,-122.388886,Parking


In [33]:
sf_venues_office.shape

(921, 7)

In [34]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Localidad').count()
    
    for n in startDf['Localidad']:
        try:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = 0

In [35]:
df_data = df.copy()
df_data.rename(columns={'Neighborhood':'Localidad'}, inplace=True)
addColumn(df_data, 'Coffee', sf_venues_coffee)
addColumn(df_data, 'High Schools', sf_venues_highschools)
addColumn(df_data, 'Universities', sf_venues_uni)
addColumn(df_data, 'Offices', sf_venues_office)
df_data.head()

,Zip Code,Localidad,Latitude,Longitude,Coffee,High Schools,Universities,Offices
3,94107,Potrero Hill,37.77,-122.39,43.0,2.0,6.0,50.0
17,94127,St. Francis Wood/Miraloma/West Portal,37.73,-122.46,8.0,3.0,1.0,45.0
15,94123,Marina,37.80,-122.44,27.0,11.0,1.0,50.0
2,94103,South of Market,37.78,-122.41,50.0,3.0,18.0,50.0
19,94132,Lake Merced,37.72,-122.48,8.0,1.0,8.0,47.0


In [36]:
df_data.shape

(20, 8)

In [40]:
weight_coffee = -1
weight_schools = 1
weight_uni = 2
weight_offices = 3

In [41]:
df_weighted = df_data[['Localidad']].copy()

In [43]:
df_weighted['Score'] = df_data['Coffee'] * weight_coffee + df_data['High Schools'] * weight_schools + df_data['Universities'] * weight_uni + df_data['Offices'] * weight_offices
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted.head()

,Localidad,Score
10,Parkside/Forest Hill,150.0
19,Lake Merced,150.0
2,South of Market,139.0
7,Ingelside-Excelsior/Crocker-Amazon,139.0
8,Castro/Noe Valley,137.0
